In [2]:
import pandas as pd
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
%run calculateSentence.ipynb
import preprocessing
pd.set_option('max_colwidth', 5000)

In [3]:
#Importazione DB

data = pd.read_csv('DB/Preprocessed_data.csv')

data = data[data['processed_title'].notna()]

data

,id,title,tags,score,post_corpus,questions,processed_title
0,742,Class views in Django,django|views|oop,0.009946,class views django view points function problem want change bit functionality yes could million keyword arguments even statements function thinking object oriented approach example page displays user page similar page displays group still similar use another data model group also members etc one way would point views class methods extend class anyone tried approach idea want share common functionality pages suggest look custom tags quite powerful also allows base template set layout page share templates fill blanks nest templates depth allowing specify layout separate groups related pages one place simply displaying data models use designed let easy show data model without write view stuff mapping url paramaters views fetching data handling edge cases rendering output etc sounds like trying combine things nt combined need different processing view depending user group object trying look use two different view functions hand common idioms want extract object detail type views perhaps could use decorator helper functions dan use django generic views easily achieve desired functionality thorough django generic views needed use class based views wanted able use full name class urlconf without always instantiate view class using helped surprisingly simple metaclass instantiate view classes use instances view functions simply point urlconf class metaclass instantiate call view class works checking first argument must actual http request would nonsense attept instantiate view class instance posted snippet always create class override function point url file instance class take look class see done,"Class views in Django view points to a function, which can be a problem if you want to change only a bit of functionality Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach For example, I have a page that displays a user This page is very similar to page that displays a group, but it's still not so similar to just use another data model Group also has members etc One way would be to point views to class methods and then extend that class Has anyone tried this approach or has any other idea?",class views django
1,2933,Create a directly-executable cross-platform GUI app using Python,user-interface|deployment|tkinter|release-management,0.009108,create directlyexecutable crossplatform gui app using python python works multiple platforms used desktop web applications thus conclude way compile executable mac windows linux problem idea start write gui anybody shed light point right direction please use basic gui development see documentation installation made command line gui pretty much reference another way develop rapid user interface write web app run locally display app browser plus go tkinter option suggested lubos hasko may want try portablepy app run windows environment without python since python installed nearly every nonwindows os default thing really need make sure nonstandard libraries use installed said possible build executables include python interpreter libraries use likely create large executable however macos x even includes support xcode ide creating full standalone gui apps run user running os x use example bring one folder files needed run app even user python pc talking windows apple os need executable file think come python without need installing create file create setuppy code save folder put program folder setuppy put folder program want make distribuitable es ps change name file anything want example run cmd folder folder right click shift choose start cmd write cmd gt python setuppy dist folder files need zip distribute pyinstaller install cmd run cmd folder file update read post make exe windows pyinstaller proper way one file images also basically python wrapper gnome toolkit found easier wrap mind around tkinter c

In [4]:
#Importazione WordEmbeddings

word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')
word_2_vec_model

In [5]:
def normalize(words):
    words = preprocessing.clear_text(words,"w2v")
    return words

In [23]:
def w2v_result(txt, tag):
    stringSearch = normalize(txt)

    #print(stringSearch)

    numberResult = "10"

    #Viene vettorizzata la query dell'utente

    vectorSearch = np.array([questionToVec(stringSearch, word_2_vec_model)])
    
    #Carico il file con gli embeddings dei titoli sui quali effettuare la corrispondenza
    allTitleEmb = pd.read_csv('DB/titleEmbeddings.csv').values

    #Calcolo della similarità del coseno for le query e tutti i titoli

    similarityCosine = pd.Series(cosine_similarity(vectorSearch, allTitleEmb)[0])
    
    #Personalizzazione della misura di similarità

    #similarityCosine = similarityCosine*(1+0.4*data.overall_scores_norm + 0.1*(data.sentiment_polarity))
    similarityCosine = similarityCosine*(1+0.4*data.score)
    
    #Lista che conterrà coppie i,j con i indice e j score di similarità
    results = []
    tmp = 0 
    
    #Se tag è null vuol dire che siamo alla prima iterazione e non dobbiamo filtrare su nessun tag
    if tag == None :   
        #Una volta calcolati gli score di similarità rendo i 10 più alti
        for i,j in similarityCosine.nlargest(int(numberResult)).iteritems():
            #print(i, data.Body[i])
            #Filtro i tag da rendere come risultato, eliminando quelli che sono già presenti nella string search per facilitare prossimi passaggi 
            tags = data.iloc[[i]].tags.item().split('|')
            #print(tags)
            for t in tags:
                if t in stringSearch:
                    tags.remove(t)
                    
            results.append(tuple([i, j, tags]))
            print(results[tmp])
            tmp = tmp+1
    #Entriamo nell'else se abbiamo il tag che deve essere incluso nei nostri risultati, 
    #e eliminiamo i record che non lo contengono dai result 
    else : 
        
        for i,j in similarityCosine.nlargest(int(numberResult)).iteritems():
            
            if tag in data.iloc[[i]].tags:
                results.append(tuple([i, j, data.iloc[[i]].tags]))
                print(results[tmp])
                tmp = tmp+1
            
    
    return results

In [24]:
#w2v_result('problem loading csv',None)

['csv', 'neo4j', 'cypher']
(31506, 0.8020832310269823, ['neo4j', 'cypher'])
['csv', 'geocode']
(41481, 0.7610547032282741, ['geocode'])
['csv', 'pandas']
(7270, 0.7584165235354433, ['pandas'])
['numpy', 'tensorflow']
(26177, 0.7456204458141819, ['numpy', 'tensorflow'])
['pandas']
(36903, 0.7402641107769051, ['pandas'])
['csv']
(17538, 0.7335108137186012, [])
['csv']
(16133, 0.733447335685383, [])
['arrays', 'numpy', 'scipy', 'mat-file']
(39442, 0.7147620451151725, ['arrays', 'numpy', 'scipy', 'mat-file'])
['csv']
(12437, 0.7098347731605613, [])
['file', 'csv']
(36160, 0.7052807481376274, ['file'])


[(31506, 0.8020832310269823, ['neo4j', 'cypher']),
 (41481, 0.7610547032282741, ['geocode']),
 (7270, 0.7584165235354433, ['pandas']),
 (26177, 0.7456204458141819, ['numpy', 'tensorflow']),
 (36903, 0.7402641107769051, ['pandas']),
 (17538, 0.7335108137186012, []),
 (16133, 0.733447335685383, []),
 (39442, 0.7147620451151725, ['arrays', 'numpy', 'scipy', 'mat-file']),
 (12437, 0.7098347731605613, []),
 (36160, 0.7052807481376274, ['file'])]